In [1]:
# https://msprime.readthedocs.io/en/stable/tutorial.html#demography

In [56]:
from math import (exp, log)
import numpy as np
import msprime

In [85]:
RHO_HUMAN = 1.6*10e-9
MU_HUMAN = 1.25*10e-8
RHO_LIMIT = (log(RHO_HUMAN)-100, log(RHO_HUMAN)+100)
MU_LIMIT = (log(MU_HUMAN)-100, log(MU_HUMAN)+100)

LENGTH_NORMALIZE_CONST = 4
ZIPPED = False
NUMBER_OF_EVENTS_LIMITS = (1, 20)
MAX_T_LIMITS = (0.01, 30)
LAMBDA_EXP = 1.0
POPULATION_LIMITS = (250, 100000)
POPULATION = 5000

IS_SEMMETRIC = True

N = 20

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [18]:
population_configurations = [
    msprime.PopulationConfiguration(
        sample_size=1, initial_size=100),
    msprime.PopulationConfiguration(
        sample_size=1, initial_size=200, growth_rate=-0.01)
]
# What's wrong with growth_rate? Why population does growth only if it's negative?

In [39]:
def create_N_population(populations_parametrs: dict) -> list:
    """
    population_parametrs: {population_name:int : [sample_size:int, initial_size:float, growth_rate:float]}
    """
    populations_configuration = []
    for population_name, parametrs in populations_parametrs.items():
        if len(parametrs)  == 3:
            sample_size, initial_size, growth_rate = parametrs
        elif len(parametrs)  == 2:
            sample_size, initial_size = parametrs
        else:
            raise f"Number ov parametrs error: expected 2 or 3, but got {len(parametrs)}"
        populations_configuration.append(
            msprime.PopulationConfiguration(
                sample_size=sample_size, 
                initial_size=initial_size, 
                growth_rate=growth_rate)
        )
    return populations_configuration

In [51]:
def create_random_migration_matrix(N: int, max_migration_rate, is_semmetric = IS_SEMMETRIC):
    migration_matrix = np.zeros(shape=(N,N))
    for i in range(N):
        for j in range(i + 1,N):
            migration_matrix[i][j] = np.random.random()*max_migration_rate
            if is_semmetric:
                migration_matrix[j][i] = migration_matrix[i][j]
            else:
                migration_matrix[j][i] = np.random.random()*max_migration_rate
    return migration_matrix

    demographic_events = [
        # CEU and CHB merge into B with rate changes at T_EU_AS
        
        msprime.MassMigration(
         time=T_EU_AS, source=2, destination=1, proportion=1.0)
            ,
        msprime.MigrationRateChange(time=T_EU_AS, rate=0),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
        # Population B merges into YRI at T_B
        msprime.MassMigration(
            time=T_B, source=1, destination=0, proportion=1.0),
        msprime.MigrationRateChange(time=T_B, rate=0),
        # Size changes to N_A at T_AF
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]

In [119]:
MAX_MIGRATION_RATE = 10

def create_MassMigration(time: float, number_of_populations: int, populations: list) -> list:
    if number_of_populations == 1:
        raise "Only one population"
    source, destination = 0,0
    while source == destination:
        source, destination = np.random.randint(number_of_populations,size=2)
    proportion = min(np.random.rand(), 1.0) # learn max of np.random.rand()
    return [msprime.MassMigration(time=time,source=source,destination=destination, proportion=proportion)]

def create_MigrationRateChange(time: float, number_of_populations: int, populations: list) -> list:
    rate = np.random.random()*MAX_MIGRATION_RATE
    i,j = np.random.randint(low=number_of_populations,size=(2))
    while i == j:
        i,j = np.random.randint(low=number_of_populations,size=(2))
    migration = [msprime.MigrationRateChange(time=time,rate=rate,matrix_index=(i,j))]
    if IS_SEMMETRIC:
        migration.append(msprime.MigrationRateChange(time=time,rate=rate,matrix_index=(j,i)))
    return migration
    
def create_PopulationParametersChange(time: float, number_of_populations: int, populations: list) -> list:
    population_id = np.random.randint(number_of_populations)
    size = int(np.random.beta(a=2, b=5) * populations[population_id])
    while size <= 0:
        size = int(np.random.beta(a=2, b=5) * populations[population_id])
    return [msprime.PopulationParametersChange(time=time, initial_size=size, population_id=population_id)]

events = {
    "MassMigration": create_MassMigration,
    "MigrationRateChange": create_MigrationRateChange,
    "PopulationParametersChange": create_PopulationParametersChange,
}

events = [create_MassMigration,create_MigrationRateChange,create_PopulationParametersChange]

def create_demographic_events(number_of_populations: int, populations: list) -> list:
    """
    """
    low, high = NUMBER_OF_EVENTS_LIMITS
    number_of_events = np.random.randint(low=low, high=high)
    times = sorted(np.random.exponential(LAMBDA_EXP, size=number_of_events))
    low, high = MAX_T_LIMITS
    max_t = np.random.uniform(low=low, high=high)
    
    alpha = 1.0
    beta = np.log(max_t + 1)/times[-1]

    def to_exp_time(time: float) -> float:
        # time -> exponentional time
        return alpha*(np.exp(beta*time) - 1)
    
    exp_times = [to_exp_time(t) for t in times]
    
    demographic_events = []
    for time in exp_times:
        event = np.random.choice(events)(time, number_of_populations, populations)
        for ev in event:
            demographic_events.append(ev)
    return demographic_events

In [120]:
create_demographic_events(2,[10,10])

[{'type': 'mass_migration', 'time': 0.007315565144512037, 'source': 1, 'dest': 0, 'proportion': 0.08289132169335423},
 {'type': 'mass_migration', 'time': 0.03768476784689545, 'source': 1, 'dest': 0, 'proportion': 0.9632225394406113},
 {'type': 'mass_migration', 'time': 0.057237492074648344, 'source': 1, 'dest': 0, 'proportion': 0.04315991195057611},
 {'type': 'mass_migration', 'time': 0.07815105517674015, 'source': 1, 'dest': 0, 'proportion': 0.2529901679892411},
 {'type': 'population_parameters_change', 'time': 0.08409354322784646, 'growth_rate': None, 'initial_size': 2, 'population': 1},
 {'type': 'mass_migration', 'time': 0.3516277799970813, 'source': 0, 'dest': 1, 'proportion': 0.5843143119231002},
 {'type': 'population_parameters_change', 'time': 0.42245538966050744, 'growth_rate': None, 'initial_size': 2, 'population': 0},
 {'type': 'migration_rate_change', 'time': 0.4972899073092327, 'rate': 1.881211597237613, 'matrix_index': (1, 0)},
 {'type': 'migration_rate_change', 'time': 0

In [121]:
np.random.randint(2)

0

In [122]:
create_random_migration_matrix(3,1,is_semmetric=False)

array([[0.        , 0.37673897, 0.39298945],
       [0.7495783 , 0.        , 0.56908147],
       [0.82916422, 0.06351183, 0.        ]])

In [123]:
dd = msprime.DemographyDebugger(
    population_configurations=population_configurations,
    migration_matrix=create_random_migration_matrix(2,0.001),
    demographic_events=create_demographic_events(2,[10,10])
)
dd.print_history()

Model =  hudson(reference_size=1)
Epoch: 0 -- 2.2700182199703445 generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0    3.68e-05 
1 |   200      205            -0.01 | 3.68e-05     0    

Events @ generation 2.2700182199703445
   - Mass migration: Lineages moved with probability 0.9131657522576428 backwards in time with source 0 & dest 1
                     (equivalent to migration from 1 to 0 forwards in time)
Epoch: 2.2700182199703445 -- inf generations
     start     end      growth_rate |     0        1    
   -------- --------       -------- | -------- -------- 
0 |   100      100                0 |     0    3.68e-05 
1 |   205      inf            -0.01 | 3.68e-05     0    



In [117]:
msprime.simulate(
        sample_size=self.sample_size,
        recombination_rate=self.recombination_rate,
        mutation_rate=self.mutation_rate,
        random_seed=self.random_seed,
        model=self.model,
        length=self.len,
        num_replicates=self.num_replicates,
        demographic_events=self.demographic_events
)

In [43]:
N = 3
N = N + 1
for i in range(N):
    for j in range(i + 1,N):
        print(i,j)

0 1
0 2
0 3
1 2
1 3
2 3


In [78]:
source, destination = np.random.randint(2,size=2) + 1

In [82]:
i,j = np.random.randint(low=2,size=(2))

In [83]:
i,j

(1, 0)

In [84]:
a = []
b = [1,2]
a.append(*b)

TypeError: append() takes exactly one argument (2 given)